# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
from torchvision.datasets import MNIST

# Set parameters and constants
Chooses between the GPU (if available) or the CPU (default) to be used for training and testing the model.

Set values for neural network hyperparameters.

Set up the transformations to be used on the training data.

In [2]:
# Choose device type (GPU or CPU), prioritizing GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
epochs = 10
batch_size = 64
learning_rate = 0.001

# Transformations for data augmentation
# Maximum rotation of 20 degrees and shift of 10% of the image's size
data_transform = transforms.Compose([
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

# Load training and testing data
We will be training and testing our model with the MNIST dataset of handwritten numbers from 0 to 9. This dataset has been already split, where 60,000 images are for training and 10,000 images for testing.

The training data will be transformed to randomly have up to 20 degrees of rotation and a maximum resizing of 10%. This will help the model deal with data that may not always be oriented correctly or have different sizes.

In [3]:
# Training dataset with data augmentation
train_dataset = MNIST(
    root='./data', train=True, transform=data_transform, download=True
)
# Testing dataset
test_dataset = MNIST(
    root='./data', train=False, transform=transforms.ToTensor(), download=True
)

# Check datasets
print('Train: ', len(train_dataset))    # 60,000
print('Test:  ', len(test_dataset))     # 10,000

# Create data loaders
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

100%|██████████| 9912422/9912422 [00:00<00:00, 140141272.27it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 42955919.80it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 43522997.80it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20074319.04it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train:  60000
Test:   10000


# Neural network
We use a Convolutional Neural Network because it's good at finding patterns in images, allowing us to classify the images.

In [4]:
# Convolutional Neural Network
class ConvNeuralNet(nn.Module):
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.softmax(x, dim=1)     # Softmax
        return x

# Train the model with the MNIST dataset

In [5]:
# Initialize model and use chosen device
model = ConvNeuralNet().to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Loop through each epoch and train the model
for each_epoch in range(epochs):
    model.train()
    for batch_id, (data, target) in enumerate(train_loader):
        data, target = torch.tensor(data).to(device), target.to(device)

        # Zeroes out gradients
        optimizer.zero_grad()
        # Model predictions
        output = model(data)
        # Loss between prediction and actual targets
        loss = loss_function(output, target)
        # Backpropagation
        loss.backward()
        # Updates the model parameters with the Adam algorithm
        optimizer.step()

        if batch_id % 100 == 0:
            print(
                f'Epoch {each_epoch + 1}/{epochs}, '
                f'Batch {batch_id}/{len(train_loader)}, '
                f'Loss: {loss.item()}'
            )

# Save the trained model
torch.save(model.state_dict(), 'mnist_model.pth')

<ipython-input-5-ef83927bc910>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = torch.tensor(data).to(device), target.to(device)


Epoch 1/10, Batch 0/938, Loss: 2.3024814128875732
Epoch 1/10, Batch 100/938, Loss: 1.997850775718689
Epoch 1/10, Batch 200/938, Loss: 1.7736642360687256
Epoch 1/10, Batch 300/938, Loss: 1.7001923322677612
Epoch 1/10, Batch 400/938, Loss: 1.6161185503005981
Epoch 1/10, Batch 500/938, Loss: 1.6424939632415771
Epoch 1/10, Batch 600/938, Loss: 1.7269768714904785
Epoch 1/10, Batch 700/938, Loss: 1.7457095384597778
Epoch 1/10, Batch 800/938, Loss: 1.6636097431182861
Epoch 1/10, Batch 900/938, Loss: 1.5906940698623657
Epoch 2/10, Batch 0/938, Loss: 1.630911946296692
Epoch 2/10, Batch 100/938, Loss: 1.5668073892593384
Epoch 2/10, Batch 200/938, Loss: 1.6112996339797974
Epoch 2/10, Batch 300/938, Loss: 1.5971479415893555
Epoch 2/10, Batch 400/938, Loss: 1.5463141202926636
Epoch 2/10, Batch 500/938, Loss: 1.6359384059906006
Epoch 2/10, Batch 600/938, Loss: 1.6689035892486572
Epoch 2/10, Batch 700/938, Loss: 1.6769788265228271
Epoch 2/10, Batch 800/938, Loss: 1.491653323173523
Epoch 2/10, Batch 9

# Load the saved model for testing

In [6]:
# Load the saved model for testing
model.load_state_dict(torch.load('mnist_model.pth'))
model.eval()

ConvNeuralNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

# Test the model for accuracy, using the test dataset

In [7]:
total_tests = 0
correct_pred = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.clone().detach().to(device), target.to(device)
        output = model(data)
        _, prediction = torch.max(output.data, 1)
        total_tests += target.size(0)
        correct_pred += (prediction == target).sum().item()

model_accuracy = correct_pred / total_tests
print(f'Accuracy on the test set: {model_accuracy * 100:.2f}%')

Accuracy on the test set: 99.11%
